This notebook is for checking ME4 Machine Learning coursework submission. You should run your model through this to check that you are getting the right answer prior to submission.

The section below is just used for clearing any previously uploaded files. You should not normally need to run it, unless you have had crashes. If you run it you will need to re-upload the datasets. You can always open the file manager on the left hand side to look at the files too.

In [ ]:
#clear all files from the work area - only really need to do this if uploading
#files multiple times
!ls -lh
!rm *.csv
!rm *.h5
!rm *.txt

This section is for uploading the two traning datasets. You can do these together if you wish by selecting both in the upload window.

In [ ]:
#upload training data sets here (run this once)
from google.colab import files

print('Upload CSV dataset files here')
uploaded = files.upload()

---
This section uploads the files selected (note -- select both .h5 and .txt for scaling parameters if you have them, otherwise it will not do any scaling), and runs the training data you have uploaded through the model. You should check that this all works OK -- you don't get any errors -- and that the fraction correct matches what you saw when you trained it. Note that this only works with one model at a time so you will need to run it twice -- once each for dataset 1 and 2.

In [5]:
#upload model files
import numpy as np
import os.path
from keras.models import load_model
import numpy as np
import pandas as pd
from keras.utils import to_categorical

from google.colab import files
print('Note: please process one model at a time - this tool is not designed to do both')
print('Upload one .h5 file and corresponding scaling file (if using one) here')
uploaded = files.upload()
h5file = ''
scalefile = ''
scale_exists = False
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  filename, file_extension = os.path.splitext(fn)
  if file_extension == '.h5':
    h5file = fn
  if file_extension == '.txt':
    scalefile = fn
    scale_exists = True
  #print("Ext: ", file_extension)
  
  filebase = filename
  
#print('File base is: ', filebase)
print('H5 file is: ', h5file)
print('Scale file is: ', scalefile)

#don't do this on Colab
# #user defined parameters (change these as necessary):
# #put your name here:
# givenname = 'Peter'
# familyname = 'Huthwaite'
# #set which dataset to use:
# dataset = 1

#filebase = familyname.lower()+'-'+givenname.lower()+'-'+np.str(dataset)

dataset = filebase[-1]
#print('Dataset considered is: ',dataset)

#load in student model
model = load_model(h5file,compile=False)

if scale_exists:
  if os.path.exists(scalefile):
      print(scalefile+' exists - loading in scaling parameters')
      scaleArray = np.loadtxt(scalefile)
  else:
      print(scalefile+' not found - assuming no scaling')
      scaleArray = np.array([np.zeros([6,]), np.ones([6,])])
else:
    print('No scale file provided - assuming no scaling')
    scaleArray = np.array([np.zeros([6,]), np.ones([6,])])

#load in the data provided to the students
df = pd.read_csv('dataset' + np.str(dataset) + '.csv')

Lt = np.array(df['Arm length (m)'][:])
Wt = np.array(df['Ball weight (kg)'][:])
Rt = np.array(df['Ball radius (mm)'][:])
Tt = np.array(df['Air temperature (deg C)'][:])
Et = np.array(df['Spring constant (N per m)'][:])
Dt = np.array(df['Device weight (kg)'][:])
Ot = np.array(df['Target hit'][:])
XtUnscaled = np.column_stack([Lt, Wt, Rt, Tt, Et, Dt])

# use values to scale validation data in XvUnscaled (whose shape is [number_of_validations,6])
Xt = (XtUnscaled-scaleArray[0,:])/scaleArray[1,:]

Yt = to_categorical(Ot)
#run the data through the model
Yt_predict = model.predict(Xt)

#output a summary of the model if you wish
model.summary()


number_correct = 0
for i in range(len(Yt)):
    if np.round(Yt[i, 0]) == np.round(Yt_predict[i, 0]):
        number_correct += 1

fraction_correct = 1.0 * number_correct / len(Yt_predict)

#import to make it bold
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

printmd("**Fraction correct with training data is: "+np.str(fraction_correct)+"**")

if fraction_correct < 0.6:
    printmd('**Warning: very poor performance on training data; likely error**')

#print('Tidying up - removing h5 file and scale file if it exists')
os.remove(h5file)
if scale_exists:
  os.remove(scalefile)

Note: please process one model at a time - this tool is not designed to do both
Upload one .h5 file and corresponding scaling file (if using one) here


Saving dataset1.csv to dataset1 (2).csv
Saving Didwania-Aman-1.h5 to Didwania-Aman-1.h5
Saving Didwania-Aman-1.txt to Didwania-Aman-1.txt
User uploaded file "dataset1.csv" with length 210961 bytes
User uploaded file "Didwania-Aman-1.h5" with length 39712 bytes
User uploaded file "Didwania-Aman-1.txt" with length 300 bytes
H5 file is:  Didwania-Aman-1.h5
Scale file is:  Didwania-Aman-1.txt
Didwania-Aman-1.txt exists - loading in scaling parameters
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 8)                 56        
_________________________________________________________________
dense_33 (Dense)             (None, 6)                 54        
_________________________________________________________________
dense_34 (Dense)             (None, 4)                 28        
_________________________________________________________________

**Fraction correct with training data is: 0.9495**